In [ ]:
import argparse
from datetime import date
import http.client as hc
from bs4 import BeautifulSoup

def init_argparser() -> argparse.ArgumentParser:
    argparser = argparse.ArgumentParser(description="Scrappy KU site for class info")
    argparser.add_argument("department", help = "department code (EECS, CLSX, ...) to look search for")
    argparser.add_argument("output", help = "file to output results to")
    return argparser

semesters = ["spring", "summer", "fall"]

def searchTermEncoder(year, semester):
    year = year % 100 # only last 2 digist
    if (semester == "spring"):
        return "4" + str(year) + "2"
    elif (semester == "summer"):
        return "4" + str(year) + "6"
    else:
        return "4" + str(year) + "9"

# https: // classes.ku.edu/Classes/CourseSearch.action?classesSearchText = eecs & searchCareer = UndergraduateGraduate & searchTerm = 4219

def traverseClasses(department : str):
    print("dull")
    

# if __name__ == "__main__":
#     argparser = init_argparser()
#     args = argparser.parse_args()
#     dept : str = args.department
#     output : str = args.output



In [ ]:
import re
def connect_url(department: str, searchTerm: str) -> str:
    url = f"https://classes.ku.edu/Classes/CourseSearch.action?classesSearchText={department}&searchCareer=UndergraduateGraduate&searchTerm={searchTerm}"
    # open initial connection
    conn = hc.HTTPSConnection("classes.ku.edu")
    conn.request("GET", url)
    resp = conn.getresponse()
    if (resp.status != 200):
        raise ConnectionError("GET failed")
    return resp.read()



In [ ]:
htmlText = connect_url("eecs", "4219")

In [94]:
def clean_text(text : str) -> str:
    text = text.replace("\n"," ").replace("\t", " ").replace("\xa0", " ").replace("(Save)"," ")
    while text != text.replace("  ", " "):
        text = text.replace("  ", " ")
    text = re.sub("^ ", "", text)
    text = re.sub(" $", "", text)
    return text

def parse_table(table) -> list[str]:
    headers = table.find_all("th")
    headers = [header.get_text() for header in headers]
    tableRows = table.find_all("tr")[1:]
    tableData = []
    for tableRow in tableRows:
        tableCols = [clean_text(tableCol.get_text()) for tableCol in tableRow.find_all("td")]
        tableData.append(tableCols)
    # The last entry of each 2 important rows are not very important
    tableData = [tableData[i] for i in range(len(tableData)) if (i % 3 == 0 or i % 3 == 1)]
    for i in range(0,len(tableData),2):
        tableData[i][1] = tableData[i][1] + "; " + tableData[i+1][1]
    tableData = [tableData[i] for i in range(0,len(tableData),2)]
    return (headers, tableData)

def parse_row(row):
    # return tuple of (className, miniDesc, description)
    descRow = row.find_next_sibling("tr")
    infoRow = descRow.find_next_sibling("tr")
    (headers, tableData) = parse_table(infoRow.find("table"))
    className = clean_text(row.find("h3").get_text())
    miniDesc = clean_text(row.find("td").get_text())
    description = clean_text(descRow.find("td").get_text())
    return (className, miniDesc, description, headers, tableData)

def parse_html(text: str):
    soup = BeautifulSoup(text, "html.parser")
    startBlocks = soup.find_all("h3")
    topRows = []
    for blk in startBlocks:
        topRows.append(blk.parent.parent)
    outStuff = {}
    for row in topRows:
        (name, miniDesc, desc, headers, tableData) = parse_row(row)
        outStuff[name] = { "name": name, "miniDesc": miniDesc, "description": desc, "tableData": headers.append(tableData) }
    return outStuff

In [95]:
import json

with open("./out.json", "w") as out_write:
    json.dump(parse_html(htmlText), out_write)
    # csvw = csv.writer(out_write, quoting=csv.QUOTE_ALL)
    # for val in parse_html(htmlText):
    #     csvw.writerow(val)

In [ ]:
parse_html(htmlText)